In [1]:
%pip install fastapi uvicorn nest-asyncio pyngrok pandas langchain langchain-community langchain-groq sentence-transformers


  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.47
    Uninstalling langchain-core-0.3.47:
      Successfully uninstalled langchain-core-0.3.47
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\lenovo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install chromadb pymongo

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.3 MB ? eta -:--:--
   -------------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\lenovo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import nest_asyncio
from pyngrok import ngrok

# Permet de relancer uvicorn dans Colab sans erreurs
nest_asyncio.apply()


In [4]:
from pymongo import MongoClient

def get_mongo_data():
    client = MongoClient("mongodb://127.0.0.1:27017/")  # Connection locale à MongoDB
    db = client["funding"]  # 🔁 Remplace par le nom de ta base MongoDB
    collection = db["campaigns"]    # 🔁 Remplace par le nom de ta collection
    documents = list(collection.find({}))
    return documents


In [5]:

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

# ✅ FastAPI app
app = FastAPI()

# ✅ Autoriser les requêtes CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # "*" permet à toutes les origines d'accéder à l'API. Tu peux spécifier l'URL de ton frontend si besoin
    allow_credentials=True,
    allow_methods=["*"],  # Autoriser toutes les méthodes HTTP (GET, POST, etc.)
    allow_headers=["*"],  # Autoriser tous les en-têtes
)

# ✅ Modèle Pydantic pour recevoir la question
class Question(BaseModel):
    question: str

# ✅ Initialisation LLM
def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key="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",  # Remplacer par variable d'env plus tard
        model_name="llama3-70b-8192"
    )

# ✅ Création de la base vectorielle
def create_vector_db_from_mongo():
    projects = get_mongo_data()

    texts = []
    for row in projects:
        full_text = f"""
        🏷️ Title: {row.get('title', '')}
        📝 Description: {row.get('description', '')}
        🏁 Goal: {row.get('goal', '')}
        💰 Amount Raised: {row.get('amountRaised', '')}
        👥 Contributors: {row.get('contributors', '')}
        📅 Created At: {row.get('createdAt', '')}
        ⏳ Days Left: {row.get('daysLeft', '')}
        🗂️ Category: {row.get('category', '')}
        🌍 Country: {row.get('country', '')}
        🧾 Type: {row.get('type', '')}
        """
        texts.append(full_text)

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = splitter.create_documents(texts)

    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")
    vector_db.persist()
    return vector_db



# ✅ Création de la QA chain
def setup_qa_chain(vector_db, llm):
    prompt_template = """
You are an AI assistant specializing in analyzing crowdfunding projects.
Your task is to generate a well-written, structured, and concise response **based only on the provided context**.

# 📌 Rules:
# - "state" = funding status: successful / failed / canceled
# - "category" = project domain
# - "goal" = funding target, "pledged" = amount raised

🔹 **Context:**
{context}

🔍 **User's Question:**
{question}
"""
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    retriever = vector_db.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain

# ✅ Lancer à la startup
llm = initialize_llm()
if not os.path.exists("./chroma_db"):
    vector_db = create_vector_db_from_mongo()
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

qa_chain = setup_qa_chain(vector_db, llm)
print("api commence")

# ✅ Route API pour poser une question
@app.post("/ask")
def ask_question(q: Question):
    try:
        result = qa_chain.run(q.question)
        return {"response": result}
    except Exception as e:
        return {"error": str(e)}
print("api finie")
# ngrok.kill()
# ngrok.set_auth_token("2w6LvW36XVZPPDrd0zLX38Xeild_4rAc1QWbReEmNzLTWbBFf")

# # Ouvrir un tunnel public sur le port 8000
# public_url = ngrok.connect(8000)
# print(f"🚀 Ton API est dispo ici : {public_url}/docs")
# import uvicorn
# uvicorn.run(app, host="localhost", port=8000)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_19844\2740881610.py:60: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')



api commence
api finie


C:\Users\lenovo\AppData\Local\Temp\ipykernel_19844\2740881610.py:62: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [9]:
# ngrok.kill()
ngrok.set_auth_token("2w6LvW36XVZPPDrd0zLX38Xeild_4rAc1QWbReEmNzLTWbBFf")

# Ouvrir un tunnel public sur le port 8000
public_url = ngrok.connect(8000)
print(f"🚀 Ton API est dispo ici : {public_url}/docs")

🚀 Ton API est dispo ici : NgrokTunnel: "https://e84b-41-225-248-197.ngrok-free.app" -> "http://localhost:8000"/docs


In [ ]:
import uvicorn
uvicorn.run(app, host="localhost", port=8000)


INFO:     Started server process [19844]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)


INFO:     ::1:65126 - "GET /docs HTTP/1.1" 200 OK
INFO:     ::1:65126 - "GET /openapi.json HTTP/1.1" 200 OK


Task exception was never retrieved
future: <Task finished name='Task-96' coro=<Server.serve() done, defined at C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\uvicorn\main.py", line 580, in run
    server.run()
  File "C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nest_asyncio.py", line 30

INFO:     ::1:49258 - "POST /ask HTTP/1.1" 200 OK
INFO:     ::1:57512 - "OPTIONS /ask HTTP/1.1" 200 OK
INFO:     ::1:57512 - "POST /ask HTTP/1.1" 200 OK
INFO:     ::1:57771 - "POST /ask HTTP/1.1" 200 OK
